In [70]:
tg.__version__

'0.0.9.9.2'

In [52]:
import pandas as pd
import json
import re
import csv
import pyTigerGraph as tg

# Notebook to explore and generate a graph for the Domain Record datasets

* Mangle data to make loading into the graph easier
* Instantiate the graph schema and graph queries (using TigerGraph - www.tigergraph.com )
* Perform community detection
* Collect community statistics to be used as features


In [2]:
# benign_whois_data.txt

!head "../data/raw/benign_whois_data.txt" 1


==> ../data/raw/benign_whois_data.txt <==
{"google.com": {"address": null, "city": null, "country": "US", "creation_date": [ "1997-09-15 04:00:00", "1997-09-15 07:00:00"    ], "dnssec": "unsigned", "domain_name": [ "GOOGLE.COM", "google.com"    ], "emails": [ "abusecomplaints@markmonitor.com", "whoisrequest@markmonitor.com"    ], "expiration_date": [ "2028-09-14 04:00:00", "2028-09-13 07:00:00"    ], "name": null, "name_servers": [ "NS1.GOOGLE.COM", "NS2.GOOGLE.COM", "NS3.GOOGLE.COM", "NS4.GOOGLE.COM", "ns1.google.com", "ns3.google.com", "ns4.google.com", "ns2.google.com"    ], "org": "Google LLC", "referral_url": null, "registrar": "MarkMonitor, Inc.", "state": "CA", "status": [ "clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited", "clientTransferProhibited https://icann.org/epp#clientTransferProhibited", "clientUpdateProhibited https://icann.org/epp#clientUpdateProhibited", "serverDeleteProhibited https://icann.org/epp#serverDeleteProhibited", "serverTransferProhibit

In [3]:
import pandas as pd
import json

b = '../data/raw/benign_whois_data.txt'

records = []
error_lines = []

# load the lines into a list, there are some error lines that needs to be dealt with
with open(b) as f: # open file
    for line in f.read().splitlines(): # for each line
        try:
            obj = json.loads(re.sub('.*}{', '{', line)) # load the line, which looks like: {"domain.name": {...values...}}
            records.append(obj)
        except:
            error_lines.append(line)
        
print(f"Number of line errors: {len(error_lines)}")  
print(f"Number of lines: {len(records)}")     


Number of line errors: 0
Number of lines: 3731


In [4]:
record_dict = {}
for item in records:
    name = [x for x in item.keys()][0]
    record_dict[name] = item[name]

In [5]:
error_list = []
for x in record_dict:
    if type(record_dict[x]) != dict:
        error_list.append(x)
        
for x in error_list:
    del record_dict[x]

In [6]:
benign_df = pd.DataFrame.from_dict(record_dict, orient='index')
benign_df.reset_index(inplace=True)

In [7]:
benign_df.head(3)

,index,address,city,country,creation_date,dnssec,domain_name,emails,expiration_date,name,...,billing_organization,billing_phone,admin_country_code,registrar_country_code,registrar_organization_id,registrar_street,registrant_handle,registrant_zip_code,billing_street,registrant_country_code
0,google.com,None,None,US,"[1997-09-15 04:00:00, 1997-09-15 07:00:00]",unsigned,"[GOOGLE.COM, google.com]","[abusecomplaints@markmonitor.com, whoisrequest...","[2028-09-14 04:00:00, 2028-09-13 07:00:00]",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,facebook.com,1601 Willow Rd,Menlo Park,US,1997-03-29 05:00:00,unsigned,FACEBOOK.COM,"[abusecomplaints@registrarsafe.com, domain@fb....",2031-03-30 04:00:00,Domain Admin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,youtube.com,None,None,US,2005-02-15 05:13:12,unsigned,"[YOUTUBE.COM, youtube.com]","[abusecomplaints@markmonitor.com, whoisrequest...","[2023-02-15 05:13:12, 2023-02-15 00:00:00]",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
pd.options.display.max_rows = 4000
print(str(list(benign_df.columns)))

['index', 'address', 'city', 'country', 'creation_date', 'dnssec', 'domain_name', 'emails', 'expiration_date', 'name', 'name_servers', 'org', 'referral_url', 'registrar', 'state', 'status', 'updated_date', 'whois_server', 'zipcode', 'email', 'fax', 'phone', 'tech_email', 'tech_fax', 'tech_name', 'tech_org', 'tech_phone', 'domain__id', 'registrant_country', 'registrant_name', 'registrant_state_province', 'registrar_id', 'registrar_url', 'admin', 'admin_application_purpose', 'admin_city', 'admin_country', 'admin_email', 'admin_fax', 'admin_id', 'admin_nexus_category', 'admin_organization', 'admin_phone', 'admin_postal_code', 'admin_state_province', 'admin_street', 'registrant_application_purpose', 'registrant_city', 'registrant_email', 'registrant_fax', 'registrant_id', 'registrant_nexus_category', 'registrant_organization', 'registrant_phone', 'registrant_postal_code', 'registrant_street', 'registrar_email', 'registrar_phone', 'tech_application_purpose', 'tech_city', 'tech_country', 'te

#############

In [9]:
import pandas as pd
import json

b = '../data/raw/malicious_whois_data.txt'

records = []
error_lines = []

# load the lines into a list, there are some error lines that needs to be dealt with
with open(b) as f: # open file
    for line in f.read().splitlines(): # for each line
        try:
            obj = json.loads(re.sub('.*}{', '{', line)) # load the line, which looks like: {"domain.name": {...values...}}
            records.append(obj)
        except:
            error_lines.append(line)
        
print(f"Number of line errors: {len(error_lines)}")  
print(f"Number of lines: {len(records)}")     


Number of line errors: 0
Number of lines: 2415


In [10]:
record_dict = {}
for item in records:
    name = [x for x in item.keys()][0]
    record_dict[name] = item[name]

In [11]:
error_list = []
for x in record_dict:
    if type(record_dict[x]) != dict:
        error_list.append(x)
        
for x in error_list:
    del record_dict[x]

In [12]:
mal_df = pd.DataFrame.from_dict(record_dict, orient='index')
mal_df.reset_index(inplace=True)

In [13]:
mal_df.head(3)

,index,address,city,country,creation_date,dnssec,domain_name,emails,expiration_date,name,...,tech_phone_ext,registrar_zip_code,registrant_type,admin_account_name,admin_company_name,admin_family_name,registrant_company_name,tech_account_name,tech_company_name,tech_family_name
0,reseptors.com,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,KN,"[2021-12-08 08:53:26, 2021-12-08T08:53:26]",unsigned,RESEPTORS.COM,domainabuse@tucows.com,"[2022-12-08 08:53:26, 2022-12-08T08:53:26]",REDACTED FOR PRIVACY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,contentcdns.net,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,KN,"[2022-02-17 23:18:15, 2022-02-17T23:18:15]",unsigned,CONTENTCDNS.NET,domainabuse@tucows.com,"[2023-02-17 23:18:15, 2023-02-17T23:18:15]",REDACTED FOR PRIVACY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,izocab.com,"220013, Belarus, Minsk, ul.YA.Kolasa, d.31, kv...",Minsk,BY,2015-12-30 00:30:35,"[unsigned, Unsigned]",IZOCAB.COM,"[abuse@reg.ru, info@brale.ru]",2022-12-30 00:30:35,Vladimir Nikolskii,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#########

In [14]:
mal_df.columns 

Index(['index', 'address', 'city', 'country', 'creation_date', 'dnssec',
       'domain_name', 'emails', 'expiration_date', 'name',
       ...
       'tech_phone_ext', 'registrar_zip_code', 'registrant_type',
       'admin_account_name', 'admin_company_name', 'admin_family_name',
       'registrant_company_name', 'tech_account_name', 'tech_company_name',
       'tech_family_name'],
      dtype='object', length=119)

In [15]:
benign_df.columns

Index(['index', 'address', 'city', 'country', 'creation_date', 'dnssec',
       'domain_name', 'emails', 'expiration_date', 'name',
       ...
       'billing_organization', 'billing_phone', 'admin_country_code',
       'registrar_country_code', 'registrar_organization_id',
       'registrar_street', 'registrant_handle', 'registrant_zip_code',
       'billing_street', 'registrant_country_code'],
      dtype='object', length=154)

In [16]:
mal_df['malicious'] = 1
benign_df['malicious'] = 0

In [17]:
combined_df = pd.concat([mal_df,benign_df], axis=0)

In [18]:
len(combined_df['index'])

6062

In [19]:
DomainRecord_cols = ['index', 'dnssec', 'name', 'malicious']

combined_df.loc[:,DomainRecord_cols].to_csv("../data/external/combined_whois_data.csv")

In [20]:
list_of_explode_columns = ['country', 'emails','whois_server', 'domain_status', 'registrar', 
                           'name_servers']

In [21]:
for _col in list_of_explode_columns:
    combined_df.loc[:,['index',_col]].explode(_col).to_csv(
    f"../data/external/combined_whois_data_{_col}.csv")

In [22]:
### Entropy data import

In [23]:
# Read in the file
with open('../data/raw/benign_entropy_data.txt', 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('{', '')
filedata = filedata.replace('}', '')
filedata = filedata.replace(':', ',')

# Write the file out again
with open('../data/external/benign_entropy_data.txt', 'w') as file:
  file.write(filedata)

In [24]:
# Read in the file
with open('../data/raw/malicious_entropy_data.txt', 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('{', '')
filedata = filedata.replace('}', '')
filedata = filedata.replace(':', ',')

# Write the file out again
with open('../data/external/malicious_entropy_data.txt', 'w') as file:
  file.write(filedata)

In [25]:
### Load IP Address Org details

import pandas as pd
import json

b = '../data/raw/benign_ip_data.txt'

records = []
error_lines = []

# load the lines into a list, there are some error lines that needs to be dealt with
with open(b) as f: # open file
    for line in f.read().splitlines(): # for each line
        try:
            obj = json.loads(re.sub('.*}{', '{', line)) # load the line, which looks like: {"domain.name": {...values...}}
            records.append(obj)
        except:
            error_lines.append(line)
        
print(f"Number of line errors: {len(error_lines)}")  
print(f"Number of lines: {len(records)}")

Number of line errors: 0
Number of lines: 3731


In [26]:
b = '../data/raw/malicious_ip_data.txt'

# load the lines into a list, there are some error lines that needs to be dealt with
with open(b) as f: # open file
    for line in f.read().splitlines(): # for each line
        try:
            obj = json.loads(re.sub('.*}{', '{', line)) # load the line, which looks like: {"domain.name": {...values...}}
            records.append(obj)
        except:
            error_lines.append(line)
        
print(f"Number of line errors: {len(error_lines)}")  
print(f"Number of lines: {len(records)}")

Number of line errors: 0
Number of lines: 6146


In [27]:
record_dict = {}
for item in records:
    name = [x for x in item.keys()][0]
    record_dict[name] = item[name]

In [28]:
error_list = []
for x in record_dict:
    if type(record_dict[x]) != dict:
        error_list.append(x)
        
for x in error_list:
    del record_dict[x]

In [29]:
org_df = pd.DataFrame.from_dict(record_dict, orient='index')
org_df.reset_index(inplace=True)

In [30]:
org_df['A_Org'] = org_df['A'].apply(pd.Series)['Org']
org_df['MX_Org'] = org_df['MX'].apply(pd.Series)['Org']

In [31]:
org_df.sample(10)

,index,A,MX,A_Org,MX_Org
5385,bsbbcqbnhweyapwrshql.com,"{'CC': 'RU', 'Org': 'REGRU-MNT'}","{'CC': 'NA', 'Org': 'NA'}",REGRU-MNT,NA
1688,sleepfoundation.org,"{'CC': 'US', 'Org': 'Cloudflare, Inc.'}","{'CC': 'US', 'Org': 'Google LLC'}","Cloudflare, Inc.",Google LLC
4137,saytt03.top,"{'CC': 'NA', 'Org': 'NA'}","{'CC': 'NA', 'Org': 'NA'}",NA,NA
3102,cityam.com,"{'CC': 'US', 'Org': 'Automattic, Inc'}","{'CC': 'US', 'Org': 'Google LLC'}","Automattic, Inc",Google LLC
182,theverge.com,"{'CC': 'US', 'Org': 'Fastly'}","{'CC': 'US', 'Org': 'Proofpoint, Inc.'}",Fastly,"Proofpoint, Inc."
1385,u-tokyo.ac.jp,"{'CC': 'NA', 'Org': 'NA'}","{'CC': 'NA', 'Org': 'NA'}",NA,NA
2621,writeablog.net,"{'CC': 'US', 'Org': 'Cloudflare, Inc.'}","{'CC': 'NA', 'Org': 'NA'}","Cloudflare, Inc.",NA
1376,lnk.to,"{'CC': 'US', 'Org': 'Amazon Technologies Inc.'}","{'CC': 'US', 'Org': 'Google LLC'}",Amazon Technologies Inc.,Google LLC
5649,warzonepw.ddns.net,"{'CC': 'NA', 'Org': 'NA'}","{'CC': 'NA', 'Org': 'NA'}",NA,NA
5936,santiagomortgages.com,"{'CC': 'NA', 'Org': 'NA'}","{'CC': 'NA', 'Org': 'NA'}",NA,NA


In [32]:
org_df.loc[org_df['MX_Org']!="NA",["index","MX_Org"]].to_csv("../data/external/combined_mx_org.csv")
org_df.loc[org_df['A_Org']!="NA",["index","A_Org"]].to_csv("../data/external/combined_A_org.csv")

In [33]:
!docker ps

CONTAINER ID   IMAGE                                     COMMAND                  CREATED       STATUS        PORTS                                                                                                                               NAMES
38e46715231e   docker.tigergraph.com/tigergraph:latest   "/bin/sh -c '/usr/sb…"   2 weeks ago   Up 43 hours   0.0.0.0:9000->9000/tcp, :::9000->9000/tcp, 0.0.0.0:14240->14240/tcp, :::14240->14240/tcp, 0.0.0.0:14022->22/tcp, :::14022->22/tcp   tigergraph


In [34]:
# To start tigergraph docker
# https://docs.tigergraph.com/tigergraph-server/current/getting-started/docker
#  docker run -d -p 14022:22 -p 9000:9000 -p 14240:14240 --name tigergraph --ulimit nofile=1000000:1000000 -v /Users/nb311848/Documents/mystuff/repos/artemis/data/external:/home/tigergraph/data -t docker.tigergraph.com/tigergraph:latest


In [53]:
conn = tg.TigerGraphConnection()

In [54]:
# print(conn.gsql('ls', options=[]))

---- Global vertices, edges, and all graphs
Vertex Types:
Edge Types:

Graphs:
- Graph Artemis(DomainRecord:v, DomainName:v, Country:v, City:v, Emails:v, Organisation:v, Nameserver:v, Registrar:v, DomainRecordStatus:v, WhoisServer:v, DomainRecord_DomainName:e, Country_City:e, DomainRecord_Country:e, DomainRecord_Emails:e, DomainRecord_Organisation:e, DomainRecord_Nameserver:e, DomainRecord_Registrar:e, DomainRecord_DomainRecordStatus:e, DomainRecord_WhoisServer:e, DomainRecord_A_Org:e, DomainRecord_MX_Org:e, co_registrar:e, co_org:e, co_nameserver:e)
Jobs:


JSON API version: v2
Syntax version: v2



In [55]:
# Clear the server as we want to reproduce teh graph creation from scratch

print(conn.gsql('drop all', options=[]))

Dropping all, about 1 minute ...
Abort all active loading jobs
Try to abort all loading jobs on graph Artemis, it may take a while ...
[ABORT_SUCCESS] No active Loading Job to abort.
Resetting GPE...
Successfully reset GPE and GSE
Stopping GPE GSE
Successfully stopped GPE GSE in 0.003 seconds
Clearing graph store...
Successfully cleared graph store
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.068 seconds
Everything is dropped.


In [56]:
#open text file in read mode
text_file = open("../src/scripts/DBImportExport_Artemis.gsql", "r")
 
#read whole file to a string
artemis_graph_gsql = text_file.read()
 
#close file
text_file.close()
 
# print(artemis_graph_gsql)

In [57]:
print(conn.gsql(artemis_graph_gsql, options=[]))

Stopping GPE GSE RESTPP
Successfully stopped GPE GSE RESTPP in 30.188 seconds
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.083 seconds
The graph Artemis is created.
Successfully created schema change jobs: [create_artemis_graph].

Current graph version 0
Trying to add vertex DomainRecord.
Trying to add vertex DomainName.
Trying to add vertex Country.
Trying to add vertex City.
Trying to add vertex Emails.
Trying to add vertex Organisation.
Trying to add vertex Nameserver.
Trying to add vertex Registrar.
Trying to add vertex DomainRecordStatus.
Trying to add vertex WhoisServer.
Trying to add edge DomainRecord_DomainName.
Trying to add edge Country_City.
Trying to add edge DomainRecord_Country.
Trying to add edge DomainRecord_Emails.
Trying to add edge DomainRecord_Organisation.
Trying to add edge DomainRecord_Nameserver.
Trying to add edge DomainRecord_Registrar.
Trying to add edge DomainRecord_DomainRecordStatus.
Trying to add edge DomainRecord_WhoisServer.
Trying t

In [58]:
loading_job = """RUN LOADING JOB load_job_whois_data{field} USING MyDataSource="/home/tigergraph/data/combined_whois_data{field}.csv\""""

In [59]:
print(conn.gsql(loading_job.format(field=""),graphname='Artemis', options=[]))

for x in list_of_explode_columns:
    print(conn.gsql(loading_job.format(field=f"_{x}"),graphname='Artemis', options=[]))                       




[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: load_job_whois_data, jobid: Artemis.load_job_whois_data.file.m1.1649225620422
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/Artemis/Artemis.load_job_whois_data.file.m1.1649225620422.log'

Job "Artemis.load_job_whois_data.file.m1.1649225620422" loading status
[WAITING] m1 ( Finished: 0 / Total: 0 )
Job "Artemis.load_job_whois_data.file.m1.1649225620422" loading status
[FINISHED] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+-----------------------------------------------------------------------------------------+
|                                     FILENAME |   LOADED LINES |   AVG SPEED |   DURATION|
|/home/tigergraph/data/combined_whois_data.csv |           6063 |     29 kl/s |     0.20 s|
+--------------------------

In [60]:
# Load entropy data
loading_job = 'RUN LOADING JOB load_job_entropy USING MyDataSource="/home/tigergraph/data/benign_entropy_data.txt"'
print(conn.gsql(loading_job, graphname='Artemis', options=[]))

loading_job = 'RUN LOADING JOB load_job_entropy USING MyDataSource="/home/tigergraph/data/malicious_entropy_data.txt"'
print(conn.gsql(loading_job, graphname='Artemis', options=[]))




[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: load_job_entropy, jobid: Artemis.load_job_entropy.file.m1.1649225659019
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/Artemis/Artemis.load_job_entropy.file.m1.1649225659019.log'

Job "Artemis.load_job_entropy.file.m1.1649225659019" loading status
[WAITING] m1 ( Finished: 0 / Total: 0 )
Job "Artemis.load_job_entropy.file.m1.1649225659019" loading status
[FINISHED] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+-----------------------------------------------------------------------------------------+
|                                     FILENAME |   LOADED LINES |   AVG SPEED |   DURATION|
|/home/tigergraph/data/benign_entropy_data.txt |           3731 |     36 kl/s |     0.10 s|
+-----------------------------------------

In [61]:
# Load MX / A org data
loading_job = 'RUN LOADING JOB load_job_A_org USING MyDataSource="/home/tigergraph/data/combined_A_org.csv"'
print(conn.gsql(loading_job, graphname='Artemis', options=[]))

loading_job = 'RUN LOADING JOB load_job_MX_org USING MyDataSource="/home/tigergraph/data/combined_MX_org.csv"'
print(conn.gsql(loading_job, graphname='Artemis', options=[]))




[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: load_job_A_org, jobid: Artemis.load_job_A_org.file.m1.1649225670045
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/Artemis/Artemis.load_job_A_org.file.m1.1649225670045.log'

Job "Artemis.load_job_A_org.file.m1.1649225670045" loading status
[WAITING] m1 ( Finished: 0 / Total: 0 )
Job "Artemis.load_job_A_org.file.m1.1649225670045" loading status
[FINISHED] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+------------------------------------------------------------------------------------+
|                                FILENAME |   LOADED LINES |   AVG SPEED |   DURATION|
|/home/tigergraph/data/combined_A_org.csv |           4435 |     21 kl/s |     0.20 s|
+------------------------------------------------------------------

In [62]:
# Install all queries

print(conn.gsql('INSTALL QUERY ALL', options=[]))

Start installing queries, about 1 minute ...
delete_co_edges query: curl -X GET 'http://127.0.0.1:9000/query/Artemis/delete_co_edges'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
tg_louvain query: curl -X GET 'http://127.0.0.1:9000/query/Artemis/tg_louvain?v_type=VALUE&e_type=VALUE&[wt_attr=VALUE]&[max_iter=VALUE]&[result_attr=VALUE]&[file_path=VALUE]&[print_info=VALUE]'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
community_stuff query: curl -X GET 'http://127.0.0.1:9000/query/Artemis/community_stuff'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
tg_label_prop query: curl -X GET 'http://127.0.0.1:9000/query/Artemis/tg_label_prop?v_type=VALUE&e_type=VALUE&max_iter=VALUE&output_limit=VALUE&[print_accum=VALUE]&[file_path=VALUE]&[attr=VALUE]'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
delete_co_loop_edges query: curl -X GET 'http://127.0.0.1:9000/query/Artemis/delete_co_loop_edges'. Add -H "Author

In [63]:
conn.graphname = 'Artemis'

In [64]:
# Run co_edge creation

print(conn.runInstalledQuery("community_stuff"))
print(conn.runInstalledQuery("delete_co_loop_edges"))

[]
[]


In [74]:
# run label propogation

# tg_label_prop (SET<STRING> v_type, SET<STRING> e_type, INT max_iter, INT output_limit, 
#  BOOL print_accum = TRUE, STRING file_path = "", STRING attr = "") 

# have to use query strings until pyTigerGrpah allows lists for sets

params = "v_type=DomainRecord&e_type=co_registrar&e_type=c_org&e_type=co_nameserver&max_iter=10000&output_limit=0&print_accum=1&attr=community"

result = conn.runInstalledQuery("tg_label_prop", params=params)

In [75]:
result

[{'@@comm_sizes_map': {'32505963': 1,
   '22020120': 1,
   '22020099': 1,
   '32505902': 1,
   '18874553': 1,
   '32505892': 1,
   '18874539': 1,
   '32505862': 1,
   '32505859': 1,
   '28311742': 1,
   '18874401': 1,
   '18874385': 1,
   '28311707': 1,
   '28311665': 1,
   '28311663': 1,
   '28311661': 1,
   '32505939': 1,
   '28311644': 1,
   '28311617': 1,
   '28311588': 1,
   '28311583': 1,
   '28311569': 1,
   '14680220': 1,
   '28311560': 1,
   '14680072': 1,
   '28311552': 1,
   '22020286': 1,
   '22020281': 1,
   '22020280': 1,
   '22020268': 1,
   '10485923': 1,
   '3145761': 1,
   '22020242': 1,
   '22020228': 1,
   '54': 1,
   '22020222': 1,
   '22020213': 1,
   '25': 1,
   '22020203': 1,
   '22020170': 1,
   '22020145': 1,
   '4194476': 1,
   '18874498': 1,
   '18874485': 1,
   '18874403': 1,
   '18874475': 1,
   '28311734': 1,
   '6291621': 1,
   '14680080': 1,
   '18874439': 1,
   '6291592': 1,
   '18874418': 1,
   '18874408': 1,
   '14680244': 1,
   '14680215': 1,
   '18

In [77]:
result = conn.runInstalledQuery("community_features_calc")
result

[{'(@@group_entropy_final)': [{'community': 7340053,
    'min_entropy': 1.58496,
    'max_entropy': 3.32193,
    'avg_entropy': 2.50532,
    'domain_count': 6,
    'count_malicious': 0},
   {'community': 13631521,
    'min_entropy': 0,
    'max_entropy': 1.58496,
    'avg_entropy': 0.86165,
    'domain_count': 3,
    'count_malicious': 0},
   {'community': 5242943,
    'min_entropy': 2.5,
    'max_entropy': 4.17574,
    'avg_entropy': 3.14201,
    'domain_count': 4,
    'count_malicious': 1},
   {'community': 1048600,
    'min_entropy': 2,
    'max_entropy': 3.49923,
    'avg_entropy': 2.87653,
    'domain_count': 4,
    'count_malicious': 0.5},
   {'community': 4194388,
    'min_entropy': 1.9183,
    'max_entropy': 2.75,
    'avg_entropy': 2.38831,
    'domain_count': 5,
    'count_malicious': 1},
   {'community': 1048665,
    'min_entropy': 2.58496,
    'max_entropy': 3.22468,
    'avg_entropy': 2.90502,
    'domain_count': 4,
    'count_malicious': 0},
   {'community': 4194345,
    